# Генерация музыки на генеративно-состязательных сетях



## Пререквизиты

### Установка зависимостей

In [1]:
!pip3 install torch matplotlib tqdm livelossplot gdown "pypianoroll>=1.0.2"

     |████████████████████████████████| 5.6MB 1.2MB/s 
     |████████████████████████████████| 61kB 3.2MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp37-none-any.whl size=5591954 sha256=c5a4e86897d2649428d890c58279f3a847c8e815c2552e5571e62a167370e69d
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
Successfully built pretty-midi


### Импорты библиотек

In [36]:
from IPython.display import clear_output
from ipywidgets import interact, IntSlider

import re
import os
import os.path
import random
from pathlib import Path
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import torch
import pypianoroll
from pypianoroll import Multitrack, Track
from tqdm import tqdm
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot

from pypianoroll import Multitrack
from pypianoroll import load as midi_load
from pypianoroll import read as midi_read

import music21 as m21
from music21 import converter,instrument, midi

from mido import MidiFile, MidiTrack, tick2second

### Подключение гугл-диска

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Настройки обучения и генерации

In [ ]:
CHECKPOINT_PATH = "/content/drive/MyDrive/models/Trance/"
MIDI_DIR = "/content/drive/MyDrive/midi_train/Trance/" 
NPZ_DIR = "/content/drive/MyDrive/npz_train/Trance/"
# Data
n_tracks = 5  # number of tracks
n_pitches = 72  # number of pitches
lowest_pitch = 24  # MIDI note number of the lowest pitch
n_samples_per_song = 8  # number of samples to extract from each song in the datset
n_measures = 4  # number of measures per sample
beat_resolution = 4  # temporal resolution of a beat (in timestep)
programs = [0, 0, 25, 33, 48]  # program number for each track
is_drums = [True, False, False, False, False]  # drum indicator for each track
track_names = ['Drums', 'Piano', 'Guitar', 'Bass', 'Strings']  # name of each track
tempo = 100

# Training
batch_size = 16
latent_dim = 128
n_steps = 20000

# Sampling
sample_interval = 100  # interval to run the sampler (in step)
n_samples = 4

In [ ]:
measure_resolution = 4 * beat_resolution
tempo_array = np.full((4 * 4 * measure_resolution, 1), tempo)
assert 24 % beat_resolution == 0, (
    "beat_resolution must be a factor of 24 (the beat resolution used in "
    "the source dataset)."
)
assert len(programs) == len(is_drums) and len(programs) == len(track_names), (
    "Lengths of programs, is_drums and track_names must be the same."
)    

## Блок подготовки модели для обучения и генерации

 ### Определяем классы генератора и дискриминатора

In [ ]:
class GeneraterBlock(torch.nn.Module):
    def __init__(self, in_dim, out_dim, kernel, stride):
        super().__init__()
        self.transconv = torch.nn.ConvTranspose3d(in_dim, out_dim, kernel, stride)
        self.batchnorm = torch.nn.BatchNorm3d(out_dim)
    
    def forward(self, x):
        x = self.transconv(x)
        x = self.batchnorm(x)
        return torch.nn.functional.relu(x)


class Generator(torch.nn.Module):
    """A convolutional neural network (CNN) based generator. The generator takes
    as input a latent vector and outputs a fake sample."""
    def __init__(self):
        super().__init__()
        self.transconv0 = GeneraterBlock(latent_dim, 256, (4, 1, 1), (4, 1, 1))
        self.transconv1 = GeneraterBlock(256, 128, (1, 4, 1), (1, 4, 1))
        self.transconv2 = GeneraterBlock(128, 64, (1, 1, 4), (1, 1, 4))
        self.transconv3 = GeneraterBlock(64, 32, (1, 1, 3), (1, 1, 1))
        self.transconv4 = torch.nn.ModuleList([
            GeneraterBlock(32, 16, (1, 4, 1), (1, 4, 1))
            for _ in range(n_tracks)
        ])
        self.transconv5 = torch.nn.ModuleList([
            GeneraterBlock(16, 1, (1, 1, 12), (1, 1, 12))
            for _ in range(n_tracks)
        ])

    def forward(self, x):
        x = x.view(-1, latent_dim, 1, 1, 1)
        x = self.transconv0(x)
        x = self.transconv1(x)
        x = self.transconv2(x)
        x = self.transconv3(x)
        x = [transconv(x) for transconv in self.transconv4]
        x = torch.cat([transconv(x_) for x_, transconv in zip(x, self.transconv5)], 1)
        x = x.view(-1, n_tracks, n_measures * measure_resolution, n_pitches)
        return x


class LayerNorm(torch.nn.Module):
    """An implementation of Layer normalization that does not require size
    information. Copied from https://github.com/pytorch/pytorch/issues/1959."""
    def __init__(self, n_features, eps=1e-5, affine=True):
        super().__init__()
        self.n_features = n_features
        self.affine = affine
        self.eps = eps
        if self.affine:
            self.gamma = torch.nn.Parameter(torch.Tensor(n_features).uniform_())
            self.beta = torch.nn.Parameter(torch.zeros(n_features))

    def forward(self, x):
        shape = [-1] + [1] * (x.dim() - 1)
        mean = x.view(x.size(0), -1).mean(1).view(*shape)
        std = x.view(x.size(0), -1).std(1).view(*shape)
        y = (x - mean) / (std + self.eps)
        if self.affine:
            shape = [1, -1] + [1] * (x.dim() - 2)
            y = self.gamma.view(*shape) * y + self.beta.view(*shape)
        return y


class DiscriminatorBlock(torch.nn.Module):
    def __init__(self, in_dim, out_dim, kernel, stride):
        super().__init__()
        self.transconv = torch.nn.Conv3d(in_dim, out_dim, kernel, stride)
        self.layernorm = LayerNorm(out_dim)
    
    def forward(self, x):
        x = self.transconv(x)
        x = self.layernorm(x)
        return torch.nn.functional.leaky_relu(x)


class Discriminator(torch.nn.Module):
    """A convolutional neural network (CNN) based discriminator. The
    discriminator takes as input either a real sample (in the training data) or
    a fake sample (generated by the generator) and outputs a scalar indicating
    its authentity.
    """
    def __init__(self):
        super().__init__()
        self.conv0 = torch.nn.ModuleList([
            DiscriminatorBlock(1, 16, (1, 1, 12), (1, 1, 12)) for _ in range(n_tracks)
        ])
        self.conv1 = torch.nn.ModuleList([
            DiscriminatorBlock(16, 16, (1, 4, 1), (1, 4, 1)) for _ in range(n_tracks)
        ])
        self.conv2 = DiscriminatorBlock(16 * 5, 64, (1, 1, 3), (1, 1, 1))
        self.conv3 = DiscriminatorBlock(64, 64, (1, 1, 4), (1, 1, 4))
        self.conv4 = DiscriminatorBlock(64, 128, (1, 4, 1), (1, 4, 1))
        self.conv5 = DiscriminatorBlock(128, 128, (2, 1, 1), (1, 1, 1))
        self.conv6 = DiscriminatorBlock(128, 256, (3, 1, 1), (3, 1, 1))
        self.dense = torch.nn.Linear(256, 1)

    def forward(self, x):
        x = x.view(-1, n_tracks, n_measures, measure_resolution, n_pitches)
        x = [conv(x[:, [i]]) for i, conv in enumerate(self.conv0)]
        x = torch.cat([conv(x_) for x_, conv in zip(x, self.conv1)], 1)
        x = self.conv2(x)
        x = self.conv3(x)          
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(-1, 256)
        x = self.dense(x)
        return x

## Генерация midi на готовых моделях

In [ ]:
def generate_midi(generator_parameters, output_filename):
  generator = Generator()
  sample_latent = torch.randn(n_samples, latent_dim)
  if torch.cuda.is_available():
      generator = generator.cuda()
      sample_latent = sample_latent.cuda()
  generator.load_state_dict(torch.load(generator_parameters))
  generator.eval()
  with torch.no_grad():
    samples = generator(sample_latent).cpu().detach().numpy()
  samples = samples.transpose(1, 0, 2, 3).reshape(n_tracks, -1, n_pitches)
  tracks = []
  for idx, (program, is_drum, track_name) in enumerate(
      zip(programs, is_drums, track_names)
  ):
      pianoroll = np.pad(
          samples[idx] > 0.5,
          ((0, 0), (lowest_pitch, 128 - lowest_pitch - n_pitches))
      )
      tracks.append(
          Track(
              name=track_name,
              program=program,
              is_drum=is_drum,
              pianoroll=pianoroll
          )
      )
  m = Multitrack(
      tracks=tracks,
      tempo=tempo_array,
      resolution=beat_resolution
  )
  m.save('out.npz')
  m1 = midi_load('out.npz')
  m1.write(output_filename)

### Пример генерации одного трека

In [ ]:
generate_midi("/content/drive/MyDrive/models/Initial/8219_check_tensor(-16.0020, device='cuda_0', grad_fn=_AddBackward0_)", "example.mid")

### Пример генерации по 5 треков на каждую модель в папках на диске models/Classical, models/Trance, models/Initial

In [ ]:
ex_path = "/content/drive/MyDrive/generated_example/Other/"
for modelfile in glob("/content/drive/MyDrive/models/Initial/*tensor*"):
  for idx in range(5):
    generate_midi(modelfile, f'{ex_path}Other_model_{modelfile.split("/")[-1].split("_")[0]}_no{idx}.mid')

ex_path = "/content/drive/MyDrive/generated_example/Classical/"
for modelfile in glob("/content/drive/MyDrive/models/Classical/*tensor*"):
  for idx in range(5):
    generate_midi(modelfile, f'{ex_path}Classical_model_{modelfile.split("/")[-1].split("_")[0]}_no{idx}.mid')

ex_path = "/content/drive/MyDrive/generated_example/Trance/"
for modelfile in glob("/content/drive/MyDrive/models/Trance/*tensor*"):
  for idx in range(5):
    generate_midi(modelfile, f'{ex_path}Trance_model_{modelfile.split("/")[-1].split("_")[0]}_no{idx}.mid')

# Нарезка, склейка midi файлов





Функции нарезки midi, склейки midi и получения midi определенной длительности из набора файлов

In [ ]:
def cut_midi(input_file, output_file, start_time, end_time) :
    """
    Cuts midi input_file from start_time to end_time and saves cutted part into 
    output_file.
    """
    input_midi = MidiFile(input_file)
    output_midi = MidiFile()
    tempo = 600000
    if input_midi.type == 2 :
        print("Can't cut the file")
        
    # Copying the time metrics between both files
    output_midi.ticks_per_beat = input_midi.ticks_per_beat

    for original_track in input_midi.tracks :
        for msg in original_track :
            if msg.type == 'set_tempo' : 
                print(msg.tempo)
                tempo = msg.tempo
                break    
    
    for original_track in input_midi.tracks :
        new_track = MidiTrack()
        total_time = 0
        for msg in original_track :
            if msg.type in ['note_on', 'note_off'] :
                total_time  += tick2second(msg.time, input_midi.ticks_per_beat, tempo)
                if total_time < start_time or total_time > end_time : continue
            new_track.append(msg)
        output_midi.tracks.append(new_track)
    
    output_midi.save(output_file)


def join_midi(input_files, output_file):
    """
    Takes list of input_files and joins it into one output_file.
    input_files - list of paths to files
    output_file - path to the output file
    """
    output_midi = MidiFile()
    tempo = 600000
    new_tracks = []
    for idx, input_file in enumerate(input_files):
      input_midi = MidiFile(input_file)
      if input_midi.type == 2 :
        continue  
      # Copying the time metrics between both files
      if idx == 0:
        output_midi.ticks_per_beat = input_midi.ticks_per_beat
        for original_track in input_midi.tracks :
          for msg in original_track :
            if msg.type == 'set_tempo' : 
              tempo = msg.tempo
              break    
      
      for track_n, original_track in enumerate(input_midi.tracks) :
        if idx == 0:
          new_tracks.append(MidiTrack())
        for msg in original_track :
          new_tracks[track_n].append(msg)
    for track in new_tracks:
      output_midi.tracks.append(track)
    
    output_midi.save(output_file)


def join_midi_to_length(input_folder, output_file, length):
  """
  Takes midi files from input folder and joins these to length seconds.
  You can use a wildcard in input folder, for example:
  "/content/1*.mid" for files in content folder which names start 
  from 1 and end with .mid 
  """
  output_midi = MidiFile()
  tempo = 600000
  new_tracks = []
  tracks_length = []
  for idx, file in enumerate(glob(input_folder)):
    input_midi = MidiFile(file)
    if input_midi.type == 2 :
        continue  
      # Copying the time metrics between both files
    if idx == 0:
      output_midi.ticks_per_beat = input_midi.ticks_per_beat
      for original_track in input_midi.tracks :
        for msg in original_track :
          if msg.type == 'set_tempo' : 
            tempo = msg.tempo
            break
    for track_n, original_track in enumerate(input_midi.tracks):
      if idx == 0:
        new_tracks.append(MidiTrack())
        tracks_length.append(0)
      for msg in original_track :
        new_tracks[track_n].append(msg)
        if msg.type in ['note_on', 'note_off'] :
          tracks_length[track_n]  += tick2second(msg.time, input_midi.ticks_per_beat, tempo)
          if tracks_length[track_n] > length : break
  for track in new_tracks:
    output_midi.tracks.append(track)
    
  output_midi.save(output_file)
           
        


In [ ]:
input_folder = "/content/drive/MyDrive/musegan/midi/*inter_bern.mid"
output_file = "/content/cut.mid"
join_midi_to_length(input_folder, output_file, 240)

In [ ]:
input_file = "/content/drive/MyDrive/musegan/midi/fake_x_bernoulli_sampling_0.npz_inf_bern.mid"
output_file = "/content/cut_2.mid"
cut_midi(input_file, output_file, start_time=10, end_time=50)

In [ ]:
input_files = ["/content/drive/MyDrive/musegan/midi/fake_x_bernoulli_sampling_0.npz_inf_bern.mid",
               "/content/drive/MyDrive/musegan/midi/fake_x_bernoulli_sampling_0.npz_inf_bern_0.mid",
               "/content/drive/MyDrive/musegan/midi/fake_x_hard_thresholding_0.npz_inf_hard.mid",
               "/content/drive/MyDrive/musegan/midi/fake_x_bernoulli_sampling_5.npz_inf_bern.mid"
              ]
output_file = "/content/join.mid"
join_midi(input_files, output_file)

# Вырезание треков из midi

In [ ]:
from mido import MidiFile, MidiTrack

def choose_tracks(input_file, output_file, tracklist):
  """
  Takes input_file and saves to output_file only tracks with indexes 
  from tracklist. 
  """
  input_midi = MidiFile(input_file)
  output_midi = MidiFile()
      
  # Copying the time metrics between both files
  output_midi.ticks_per_beat = input_midi.ticks_per_beat
  
  for idx, original_track in enumerate(input_midi.tracks):
    if idx in tracklist:
      new_track = MidiTrack()
      for msg in original_track :
          new_track.append(msg)
      output_midi.tracks.append(new_track)
  
  output_midi.save(output_file)  

In [ ]:
choose_tracks("/content/2UNLIMITED - Are You Ready For This.mid", "cutted.mid", [0, 1, 2, 6])

# Тональное инвертирование midi

In [ ]:
from mido import MetaMessage
from mido import MidiFile
from mido import MidiTrack


def check_tones(input_file):
  """
  Returns tuple with maximum and minumum note number 
  """
  try:
    mid = MidiFile(input_file)
  except:
    return None
  min_note = 128
  max_note = 0
  for track in mid.tracks:
    for message in track:
      if 'note' in dir(message):
        if message.note < min_note:
          min_note = message.note
        if message.note > max_note:
          max_note = message.note
  return (min_note, max_note)

def tone_invert(input_file, output_file, basenote=50):
  """
  Inverts tones over basenote and saves to output_file.
  Optimal basenote is (max_note + min_note)/2
  """  
  try:
    mid = MidiFile(input_file)
  except:
    return None
  inverted = MidiFile()

  for track in mid.tracks:
    new_track = MidiTrack()
    if 'drum' in track.name.lower():
      new_track = track
    else:
      for message in track:
        if isinstance(message, MetaMessage):
          new_track.append(message)
        else:
          if 'note' in dir(message):
            inverted_note = basenote - (message.note - basenote)
            new_track.append(message.copy(note=inverted_note, time=int(message.time)))
          else:
            new_track.append(message)

    inverted.tracks.append(new_track)
  try:
    inverted.save(output_file)
  except:
    pass

In [ ]:
min_note, max_note = check_tones("/content/Ievan_Polkka.mid")
tone_invert("/content/Ievan_Polkka.mid", "inverted.mid", (max_note + min_note)/2)

In [ ]:
in_dir = "/content/drive/MyDrive/midi_collection/source/*"
out_dir = "/content/drive/MyDrive/midi_collection/inverted/"

for dir in glob(in_dir):
  try:
    os.mkdir(out_dir + dir.split("/")[-1])
  except BaseException:
    pass
  
  for idx, song in enumerate(Path(dir).glob('**/*.mid')):
    print(song)
    min_note, max_note = check_tones(song)
    tone_invert(song, out_dir + dir.split("/")[-1] + f"/{idx:06}.mid", (max_note + min_note)/2)

# Ракоход midi (обратное проигрывание)

In [ ]:
from mido import MetaMessage
from mido import MidiFile
from mido import MidiTrack


def invert_midi(input_file, output_file):
    """
    Takes input_file, inverts its playback and saves to output_file
    """
    output_midi = MidiFile()
    tempo = 600000
    new_tracks = []
    try:
      input_midi = MidiFile(input_file)
    except:
      return None
    # Copying the time metrics between both files
    
    output_midi.ticks_per_beat = input_midi.ticks_per_beat
    for original_track in input_midi.tracks :
      for msg in original_track :
        if msg.type == 'set_tempo' : 
          tempo = msg.tempo
          break    
      
      for track_n, original_track in enumerate(input_midi.tracks) :
        new_tracks.append(MidiTrack())
        for msg in original_track :
          new_tracks[track_n].append(msg)
    for track in new_tracks:
      output_midi.tracks.append(track[::-1])
    
    try:
      output_midi.save(output_file)
    except:
      pass

In [ ]:
invert_midi("/content/scooter-let_me_be_your_valentine.mid", "inverted.mid")

In [ ]:
in_dir = "/content/drive/MyDrive/midi_collection/source/*"
out_dir = "/content/drive/MyDrive/midi_collection/reversed/"

for dir in glob(in_dir):
  try:
    os.mkdir(out_dir + dir.split("/")[-1])
  except BaseException:
    continue
  
  for idx, song in enumerate(Path(dir).glob('**/*.mid')):
    print(song)
    invert_midi(song, out_dir + dir.split("/")[-1] + f"/{idx:06}.mid")

/content/drive/MyDrive/midi_collection/source/The Sound Of The 80's (Engelstalig)/000418.mid
/content/drive/MyDrive/midi_collection/source/The Sound Of The 80's (Engelstalig)/000419.mid
/content/drive/MyDrive/midi_collection/source/The Sound Of The 80's (Engelstalig)/000420.mid
/content/drive/MyDrive/midi_collection/source/The Sound Of The 80's (Engelstalig)/000421.mid
/content/drive/MyDrive/midi_collection/source/The Sound Of The 80's (Engelstalig)/000422.mid


# Перемешивание midi

In [59]:
import random


def open_midi(midi_path, remove_drums=False):
  """
  Opens midi file from midi_path as a music21.Stream object.
  """
  mf = m21.midi.MidiFile()
  mf.open(midi_path)
  mf.read()
  mf.close()
  if (remove_drums):
    for i in range(len(mf.tracks)):
      mf.tracks[i].events = [ev for ev in mf.tracks[i].events if ev.channel != 10]          
  return m21.midi.translate.midiFileToStream(mf)


def shuffle_midi(in_file, out_file):
  """
  Shuffles measures in a midi file
  """
  s_in = open_midi(in_file)
  try:
    midi_measures = s_in.measures(1, 9999999)
  except:
    print("Can't convert this!")
    return 0
  s_out = m21.stream.Score()
  
  for part in midi_measures:
    partnew = m21.stream.Part()
    sample_part = random.sample(list(part), len(part))
    for el in sample_part:
      partnew.append(el)
    s_out.append(partnew)
  
  s_out.write('midi', out_file) 
  

In [64]:

def shuffle_midi(in_file, out_file):
  """
  Shuffles measures in a midi file
  """
  s_in = open_midi(in_file)
  try:
    midi_measures = s_in.measures(1, 9999999)
  except:
    print("Can't convert this!")
    return 0
  s_out = m21.stream.Score()
  num_measures = max([len(part) for part in midi_measures])
  measure_sample = random.sample(range(num_measures), num_measures)
  for part in midi_measures:
    partnew = m21.stream.Part()
    part_lst = list(part)
    for idx in measure_sample:
      try:
        partnew.append(part_lst[idx])
      except IndexError:
        continue
    #sample_part = random.sample(list(part), len(part))
    #for el in sample_part:
    #  partnew.append(el)
    s_out.append(partnew)
  
  s_out.write('midi', out_file) 

In [65]:
shuffle_midi("/content/george thorogood - bad to the bone.mid", "out.mid")

# Замена нот на аккорды

In [ ]:
def notes_to_chords(input_file, output_file, tracks=[1, 2, 3, 4, 5], chords="major") :
    """
    Changes single notes in midi to chords.
    input_file  - path to the input midi file
    output_file - path to the output file to save
    tracks - list of tracks numers to modify
    chords - "major" or "minor" for corresponding triads
    """
    input_midi = MidiFile(input_file)
    output_midi = MidiFile()

    # Copying the time metrics between both files
    output_midi.ticks_per_beat = input_midi.ticks_per_beat

    for track_n, original_track in enumerate(input_midi.tracks):
      if track_n in  tracks:
        new_track = MidiTrack()
        cur = []
        for idx, msg in enumerate(original_track):
          if msg.type == "note_on":
            cur.append(msg)
          elif msg.type == "note_off":
            cur.append(msg)
            if len(cur) != 2:
              for el in cur:
                new_track.append(el)
              cur=[]
            else:
              note = cur[0].note
              if chords == "major":
                new_track.append(cur[0].copy())
                new_track.append(cur[0].copy(note=note+4, time=0))
                new_track.append(cur[0].copy(note=note+7, time=0))
                new_track.append(cur[1])
                new_track.append(cur[1].copy(note=note+4, time=0))
                new_track.append(cur[1].copy(note=note+7, time=0))
              elif chords == "minor":
                new_track.append(cur[0].copy())
                new_track.append(cur[0].copy(note=note+3, time=0))
                new_track.append(cur[0].copy(note=note+7, time=0))
                new_track.append(cur[1])
                new_track.append(cur[1].copy(note=note+3, time=0))
                new_track.append(cur[1].copy(note=note+7, time=0))
              cur=[]
          else:
            new_track.append(msg)
        output_midi.tracks.append(new_track)

    output_midi.save(output_file)

In [ ]:
input_file = "31.mid"
output_file = "out.mid"
notes_to_chords(input_file, output_file, [1], "minor")

# Изменение инструментов

Определяем функцию изменения инструментов, список инструментов подается в виде списка объектов[ music21.instrument](https://web.mit.edu/music21/doc/moduleReference/moduleInstrument.html)


In [35]:
def open_midi(midi_path, remove_drums=False):
  """
  Opens midi file from midi_path as a music21.Stream object.
  """
  mf = m21.midi.MidiFile()
  mf.open(midi_path)
  mf.read()
  mf.close()
  if (remove_drums):
    for i in range(len(mf.tracks)):
      mf.tracks[i].events = [ev for ev in mf.tracks[i].events if ev.channel != 10]          
  return m21.midi.translate.midiFileToStream(mf)


def change_instruments(input_file, output_file, new_instruments):
  """
  Opens midi file from input_file, change its instruments to new_instruments. 
  The new instruments should be a list with 5 music21.instrument.Instrument objects. 
  For example instrument.Piano(), instrument.Bass() and so on.
  Function saves file in output_file path.
  """
  s = open_midi(input_file)
  for i, part in enumerate(s):
    for el in part.recurse():
      if isinstance(el, instrument.Instrument):
        try:
          el.activeSite.replace(el, new_instruments[i])
        except BaseException:
          el.activeSite.replace(el, instrument.Piano())

  s.write('midi', output_file)

Пример использования функции изменения инструментов

In [ ]:
input_file = "/content/drive/MyDrive/midi2wav/midi/fake_x_hard_thresholding_0.npz_inter_hard.mid"
output_file = "/content/drive/MyDrive/midi2wav/midi/__fake_x_hard_thresholding_0.npz_inter_hard.mid"
new_instruments = [instrument.SnareDrum(), instrument.Piano(), instrument.ElectricBass(), instrument.ElectricGuitar(), instrument.Guitar()]

change_instruments(input_file, output_file, new_instruments)

In [ ]:
from mido import MetaMessage
from mido import MidiFile
from mido import MidiTrack

def change_instruments_to_piano(input_file, output_file):
  """
  Changes every instrument to piano, except the drums
  """  
  try:
    mid = MidiFile(input_file)
  except:
    return None
  out = MidiFile()
  tempo = 600000
  new_tracks = []
    
  # Copying the time metrics between both files
    
  out.ticks_per_beat = mid.ticks_per_beat
  for original_track in mid.tracks :
    for msg in original_track :
      if msg.type == 'set_tempo' : 
        tempo = msg.tempo
        break    


  for track in mid.tracks:
    new_track = MidiTrack()
    for message in track:
      if message.type == 'program_change':
        if message.program != 10:
          new_track.append(message.copy(program=0))
          continue
      new_track.append(message)
    out.tracks.append(new_track)
  try:
    out.save(output_file)
  except:
    pass

In [ ]:
change_instruments_to_piano("/content/scooter-let_me_be_your_valentine.mid", "out.mid")

In [ ]:
in_dir = "/content/drive/MyDrive/Жанры/*"
out_dir = "/content/drive/MyDrive/midi_collection/source/"

for dir in glob(in_dir):
  try:
    os.mkdir(out_dir + dir.split("/")[-1])
  except BaseException:
    continue
  print(dir)
  for idx, song in enumerate(Path(dir).glob('**/*.mid')):
    print(song)
    change_instruments_to_piano(song, out_dir + dir.split("/")[-1] + f"/{idx:06}.mid")


/content/drive/MyDrive/Жанры/Поп
/content/drive/MyDrive/Жанры/Поп/Abba/The-Winner-Takes-It-All-2.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Money-Money-Money-5.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Honey-Honey.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Knowing-Me-Knowing-You-2.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Abba-Nr-2-(Medley).mid
/content/drive/MyDrive/Жанры/Поп/Abba/Fernando-2.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Andante-Andante.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Thank-You-For-The-Music-1.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Take-A-Change-On-Me-1.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Supertrouper-2.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Intermezzo-Nr.1.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Waterloo-2.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Abba-Medley-(Stars-On-45)-1.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Fernando-4.mid
/content/drive/MyDrive/Жанры/Поп/Abba/Abba-(Medley)-2.mid
/content/drive/MyDrive/Жанры/Поп/Abba/I-Do-I-Do-I-Do-1.mid
/con

# Конвертация midi в wav с использованием различных звуковых шрифтов

Устанавливаем fluidsynth

In [ ]:
!apt-get install fluidsynth

Определяем функцию конвертации.

In [ ]:
from midi2audio import FluidSynth


def convert_midi_to_wav(input_file, output_file, soundfont=""):
  """
  Converts midi file from input_file to wav output_file with particulary soundfont.
  You have to install fluidsynth utility first. The soundfont waits for 
  a whole path to sf2 file.
  """
  fs = FluidSynth(soundfont)
  fs.midi_to_audio(input_file, output_file)

Пример использования функции конвертации midi в wav. Должен быть подключен Google Drive.

In [ ]:
sf = "/content/drive/MyDrive/midi2wav/soundfonts/Anologue Heaven.SF2"
input_file = "/content/drive/MyDrive/midi2wav/midi/fake_x_hard_thresholding_9.npz_inter_hard.mid"
output_file = "/content/drive/MyDrive/midi2wav/wav/test1.wav"
convert_midi_to_wav(input_file, output_file, sf)

# Переименование шрифтов по порядку номеров

In [31]:
def rename_files(input_folder):
  file_list = []
  nums = []
  pattern=r".*kit(\d+).sf2$"
  for filename in glob(input_folder + "*.SF2") + glob(input_folder + "*.sf2"):
    mtch = re.search(pattern, filename.lower())
    if mtch:
      nums.append(int(mtch.group(1)))
    else:
      file_list.append(filename)
  idx = max(nums) + 1
  for filename in file_list:
    os.rename(filename, input_folder + f"kit{idx}.sf2")
    idx += 1


In [34]:
rename_files("/content/drive/MyDrive/midi2wav/soundfonts/")